# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

**IT'S FINALLY TIME!**

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time for me to apply my studies.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between being a challenge while also being an approachable task. Let's begin!

(I promise my documentation will be more formal as these examples increase in complexity).

# Load and Examine Data

In [183]:
import pandas as pd
import re
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt

In [184]:
cars = pd.read_csv("data/used_cars.csv")

Let's just get a quick overview of some basic precleaning stuff.

In [185]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [186]:
print(f"Missing Values:\n{cars.isna().sum()}\n")
print(f"Total Duplicates:\n{cars.duplicated().sum()}")

Missing Values:
brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

Total Duplicates:
0


For simplicity, we're gonna just remove the missing data. We can look into accounting for missing values when my initial models are finished.

In [187]:
cars.dropna(inplace=True)

Now that that's out of the way, let's actually take a peak at the data now.

In [188]:
cars

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
...,...,...,...,...,...,...,...,...,...,...,...,...
4003,Mercedes-Benz,E-Class E 300 4MATIC,2018,"53,705 mi.",Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,Yes,"$25,900"
4004,Bentley,Continental GT Speed,2023,714 mi.,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,"$349,950"
4005,Audi,S4 3.0T Premium Plus,2022,"10,900 mi.",Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,"$53,900"
4007,Ford,F-150 Raptor,2020,"33,000 mi.",Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,"$62,999"


# Clean Data

Now that the preliminary stuff is out of the way, let's start cleaning the data up.

It'd be best to start simple here and worry about the simple numeric conversion first. We'll start by removing unnecessary string values from columns that will be designated as numeric. We'll also use `float64` values for the initial regression attempts, then check if there's any difference in the results using `float32`.

After that, we can look into splitting the columns and determining the best way to structure the rest of the data.

In [ ]:
# Remove non-integer values from numeric columns
cars[["milage", "price"]] = cars[["milage", "price"]].replace(r'[^\d]', '', regex=True)    

# Split `engine` column for greater detail
cars["horsepower"] = cars["engine"].str.extract(r'([\d.]+)HP', expand=False)
cars["liters"] = cars["engine"].str.extract(r'([\d.]+)L', expand=False)
cars["cylinders"] = cars["engine"].str.extract(r'([\d.]+)V or ')
cars["type"] = # Designate cars as electric to group by electric and gas later. In fact... we might want to run two different regressions for electric and gas!

# Simplify `transmission` column
# Make sure to split into manual, automatic, and dual shift. This will help a great deal I think, as these designations are huge indicators of a car price
cars["transmission"] = cars[""] # Convert to 

cars

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,liters,cylinders
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300,300.0,3.7,NaN
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005,NaN,3.8,NaN
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500,354.0,3.5,NaN
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,31000,292.0,2.0,NaN
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,7300,282.0,4.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4003,Mercedes-Benz,E-Class E 300 4MATIC,2018,53705,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,Yes,25900,241.0,2.0,NaN
4004,Bentley,Continental GT Speed,2023,714,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,349950,NaN,6.0,NaN
4005,Audi,S4 3.0T Premium Plus,2022,10900,Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,53900,349.0,3.0,NaN
4007,Ford,F-150 Raptor,2020,33000,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,62999,450.0,3.5,NaN
